In [3]:
from IPython.display import display

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import sys
from pathlib import Path
import pickle
from time import time
from tqdm.notebook import tqdm

In [10]:
def show_df(df: pd.DataFrame):
    display(df.head())
    print(df.shape)

In [46]:
import re

arr = []

for dir in ['ecoli29x', 'ecoli100x']:
    path = Path('outputs')/dir
    for filename in path.iterdir():
        g = re.match(
            r'^outputs/.*/(opt_injection|one2all|injection)_N_([12])_n_([0-9]+)_gpu_-?([1-4]).out$',
            str(filename)
        )
        if g is None:
            continue

        job=g.group(1)
        N=int(g.group(2))
        n=int(g.group(3))*N
        gpu=int(g.group(4))*N

        # print(job, N,n,gpu)

        main_time = 0
        align_time = 0

        with open(filename, 'r') as f:
            flag = False
            for line in f.readlines():
                if not flag:
                    if line == 'INFO: ELBA runtimes:\n':
                        flag = True
                else:
                    if (g := re.search(r'^  Main:([0-9]+\.[0-9]+) ms',line)):
                        main_time = float(g.group(1))
                    elif line.startswith('  Main:DprAlign():'):
                        align_time = float(line[len('  Main:DprAlign():'):-4])

        if main_time == 0 or align_time == 0:
            continue

        arr.append({
            'dataset': dir,
            'scheduler': job,
            'N': N,
            'n': n,
            'gpu': gpu,
            'total_time': main_time/1000,
            'align_time': align_time/1000,
            'delta_time': (main_time-align_time)/1000
        })

df_data = pd.json_normalize(arr).sort_values(
    by=['dataset', 'scheduler', 'gpu', 'n', 'N'],
    ignore_index=True
)

show_df(df_data)

,dataset,scheduler,N,n,gpu,total_time,align_time,delta_time
0,ecoli100x,injection,1,16,1,102.795418,59.332869,43.462549
1,ecoli100x,injection,1,16,2,76.602072,33.510017,43.092055
2,ecoli100x,injection,2,16,2,97.245425,55.569178,41.676247
3,ecoli100x,injection,1,1,4,798.665151,61.743269,736.921881
4,ecoli100x,injection,1,4,4,156.315297,16.127364,140.187933


(62, 8)
